In [1]:
import numpy as np

In [2]:
# Ax = b

A = np.array([
    [2, 1, 0, 0],
    [3/13, 2, 10/13, 0],
    [0, 1/2, 2, 1/2],
    [0, 0, 1, 2]
])
b = np.array([
    -46.6666,
    -4.00002,
    -2.7000,
    -17.4000
])

In [3]:
x = np.linalg.solve(A, b)
x

array([-23.53131149,   0.39602297,   0.82970772,  -9.11485386])

### spline interpolation

In [4]:
def f0_k(list_x, list_y):
    k = len(list_x)
    if k == 1:
        return list_y[0]
        
    xk = list_x[-1]
    x0 = list_x[0]
    f1 = f0_k(list_x[1:k], list_y[1:k])
    f2 = f0_k(list_x[0:k-1], list_y[0:k-1])
    coef = (f1 - f2) / (xk - x0)
    #print(f"f1={f1:.6f}, f2={f2:.6f}, xk={xk:.6f}, x0={x0:.6f}, coef={coef:.6f}")
    return coef

In [5]:
def spline_interpolation_condition_1(list_x, list_y, dy0, dyn):

    n = len(list_x) - 1  # number of hj

    # h_j
    list_h = [list_x[j+1] - list_x[j] for j in range(n)]
    print("list_h", list_h)

    # mu_j, let m0 be undefined
    list_mu = [ list_h[j-1] / (list_h[j-1] + list_h[j]) for j in range(1,n) ]
    list_mu = list_mu + [1]
    print("list_mu", list_mu)
    
    # lambda_j
    list_lambda = [ list_h[j] / (list_h[j-1] + list_h[j]) for j in range(1,n) ]
    list_lambda = [1] + list_lambda
    print("list_lambda", list_lambda)

    # d_j
    list_d = []
    for j in range(n-1):
        dj = 6 * f0_k(list_x[j:j+3], list_y[j:j+3])
        list_d.append(dj)
    d0 = (f0_k(list_x[0:2], list_y[0:2]) - dy0) * 6 / list_h[0]
    dn = (dyn - f0_k(list_x[-2:], list_y[-2:])) * 6 / list_h[-1]
    list_d = [d0] + list_d + [dn]
    print("list_d", list_d)

    # construct Ax = b
    m = n+1
    A = np.zeros([m, m])
    for i in range(m):
        for j in range(m):
            if i == j:
                A[i,j] = 2
            elif i+1 == j:
                A[i,j] = list_lambda[i]
            elif i-1 == j:
                A[i,j] = list_mu[j]
            else:
                continue
    print(A)

    list_M = np.linalg.solve(A, np.array(list_d)).tolist()
    print("list_M", list_M)

    def S1(x):
        for j in range(n):
            if x < list_x[j+1]:
                a0 = list_M[j] / (6 * list_h[j])
                a1 = list_M[j+1] / (6 * list_h[j])
                a2 = (list_y[j] - list_M[j] * list_h[j]**2 / 6) / list_h[j]
                a3 = (list_y[j+1] - list_M[j+1] * list_h[j]**2 / 6) / list_h[j]
                print(f"a0={a0:.6f}, a1={a1:.6f}, a2={a2:.6f}, a3={a3:.6f}")
                sx = a0 * (list_x[j+1]-x)**3 + a1 * (x - list_x[j])**3
                sx += a2 * (list_x[j+1]-x) + a3 * (x - list_x[j])
                return sx
    return S1

In [6]:
list_x = [27.7, 28, 29, 30]
list_y = [4.1, 4.3, 4.1, 3.0]
dy0 = 3
dyn = -4

In [7]:
S1 = spline_interpolation_condition_1(list_x, list_y, dy0, dyn)

list_h [0.3000000000000007, 1, 1]
list_mu [0.2307692307692312, 0.5, 1]
list_lambda [1, 0.7692307692307688, 0.5]
list_d [-46.66666666666658, -3.999999999999994, -2.6999999999999984, -17.400000000000002]
[[2.         1.         0.         0.        ]
 [0.23076923 2.         0.76923077 0.        ]
 [0.         0.5        2.         0.5       ]
 [0.         0.         1.         2.        ]]
list_M [-23.531353135313488, 0.39603960396039933, 0.8297029702970301, -9.114851485148517]


In [8]:
S1(27.8)

a0=-13.072974, a1=0.220022, a2=14.843234, a3=14.313531


4.295636230289697